
# 📊 Business Intelligence Dashboard (Notebook Version)

Ce notebook permet :
- de charger le **Data Warehouse**
- de calculer les **KPI principaux**
- de visualiser les données avec **Plotly**
- sans lancer un serveur Dash (exécution directe dans Jupyter)

---


## 1️⃣ Import des librairies

In [ ]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


: 

## 2️⃣ Chargement des données du Data Warehouse

In [ ]:

fact = pd.read_csv("data/warehouse/fact_orders.csv")
dim_customer = pd.read_csv("data/warehouse/dim_customers.csv")
dim_employee = pd.read_csv("data/warehouse/dim_employees.csv")
dim_time = pd.read_csv("data/warehouse/dim_temps.csv")


## 3️⃣ Nettoyage des noms de colonnes

In [ ]:

for df in [fact, dim_customer, dim_employee, dim_time]:
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")


## 4️⃣ Création des colonnes descriptives

In [ ]:

dim_customer['customer_and_company'] = dim_customer['customerid'].astype(str) + " - " + dim_customer['companyname']
dim_employee['employee_name'] = dim_employee['firstname'] + " " + dim_employee['lastname']


## 5️⃣ Construction du schéma en étoile

In [ ]:

fact = fact.merge(
    dim_customer[['customer_key', 'customer_and_company', 'region', 'customerid']],
    on='customer_key', how='left'
)

fact = fact.merge(
    dim_employee[['employee_key', 'employee_name']],
    on='employee_key', how='left'
)

fact = fact.merge(
    dim_time[['date_key', 'date', 'day', 'month', 'year']],
    left_on='orderdate_key', right_on='date_key', how='left'
)


## 6️⃣ KPI Globaux

In [ ]:

fact['status'] = fact['delivered'].apply(lambda x: "Livré" if x == 1 else "Non livré")

total = len(fact)
liv = fact['delivered'].sum()
non = total - liv
taux = round((liv / total * 100), 2)

print(f"Total commandes : {total}")
print(f"Livrées : {liv}")
print(f"Non livrées : {non}")
print(f"Taux de livraison : {taux}%")


## 7️⃣ Visualisations

In [ ]:

px.bar(
    fact,
    x="date",
    title="Commandes au fil du temps"
).show()


In [ ]:

px.histogram(
    fact,
    x="employee_name",
    title="Commandes par Employé"
).show()


In [ ]:

px.histogram(
    fact,
    x="customer_and_company",
    title="Commandes par Client"
).show()


In [ ]:

px.pie(
    fact,
    names="status",
    title="Livré / Non Livré"
).show()


In [ ]:

px.pie(
    fact,
    names="region",
    title="Répartition des livraisons par région"
).show()


## 8️⃣ Cube OLAP 3D

In [ ]:

cube = go.Figure()

cube.add_trace(go.Scatter3d(
    x=fact["employee_name"],
    y=fact["customer_and_company"],
    z=fact["date"],
    mode="markers",
    marker=dict(
        size=6,
        color=fact["status"].map({"Livré": "green", "Non livré": "red"}),
        opacity=0.8
    ),
    text=(
        "Employé : " + fact["employee_name"] + "<br>" +
        "Client : " + fact["customer_and_company"] + "<br>" +
        "Date : " + fact["date"].astype(str) + "<br>" +
        "Statut : " + fact["status"]
    ),
    hoverinfo="text"
))

cube.update_layout(
    title="Cube OLAP 3D : Employé × Client × Date",
    scene=dict(
        xaxis_title="Employé",
        yaxis_title="Client",
        zaxis_title="Date"
    )
)

cube.show()
